**Import Packages**

In [ ]:
# import packages

import numpy as np
import pandas as pd
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.decomposition import PCA
import scipy.io as sio
import matplotlib as mpl
from pathlib import Path
from collections import Counter
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import merge
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.models import Model
from keras.layers import Input, Dense, concatenate,multiply
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.layers import Lambda
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Add
from keras.layers import LayerNormalization





**Classification Methods**

Label Discretization and Majority Voting


In [ ]:
#function to convert continous labels into binary labels
def bin_labels(data_rec):             
    count_0 = 0
    count_1 = 0
    median_value = np.median(data_rec)
    print("Inside bin labels function: " + str(median_value))
    for it in range(0,len(data_rec),1):
        if data_rec[it]<=median_value :
            count_0 += 1
            data_rec[it]=0
        else:
            count_1 += 1
            data_rec[it]=1
    print(count_0, count_1)
    return data_rec
def find_median(data_rec):
    median_value = np.median(data_rec)
    print("Inside med function: " + str(median_value))
    return median_value

#function to convert continous labels into binary labels for the test set based on the median from train data
def bin_labels_test(data_rec, train_median):             
    count_0 = 0
    count_1 = 0
    # median_value = train_median
    for it in range(0,len(data_rec),1):
        if data_rec[it]<=train_median :
            count_0 += 1
            data_rec[it]=0
        else:
            count_1 += 1
            data_rec[it]=1
    print(count_0, count_1)
    return data_rec 


#function to return the majority from a list of labels
def majority_vote(arr):
    vote_count = Counter(arr)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        # It is a tie
        return 0
    return top_two[0][0]



**Creating Kineme Feature Chunks**

In [ ]:
# onehot encoding of kineme sequence
def onehot_encoding(ks, nKineme):
    #print(ks)
    onehot_encoded = list()
    for k in ks:
        vec = [0 for _ in range(nKineme)]
        vec[k-1] = 1
        onehot_encoded.append(vec)
    return onehot_encoded



def ks_encoding(ks, nKineme):
    # ks is a numpy ndarray
    m, n = ks.shape #m=92, n=29
    #print(m, n)
    ks = ks.tolist() #converted to list
    encoded_features = np.asarray(
        [np.asarray(onehot_encoding(ks[i], nKineme)) for i in range(m)]
    )
    return encoded_features

def data_preprocess(chunk_time, input_file_kineme, label_val, num_chunk): 
    data_list = []
    total_num_chunks = 0
    label_file_index = 0
    final_label = [] 
    final_data_file = input_file_kineme.T     
    for num in range(0, num_chunk):
        data_chunk = final_data_file[num*(chunk_time-1):((num+1)*(chunk_time-1))]
        # print("Chunks dim in kineme " + str(num*(chunk_time-1)) + " "+ str(((num+1)*(chunk_time-1))))
        data_chunk = data_chunk.to_numpy()
        data_list.append(data_chunk.flatten())
    one_list = label_val.repeat(num_chunk)
    final_label.extend(one_list)                 
    return  data_list, final_label

#Function to create kineme data matrix for Test Data
def data_preprocess_test(chunk_time, input_file_kineme, num_chunk): 
    data_list = []
    total_num_chunks = 0
    label_file_index = 0
    final_label = []
    final_data_file = input_file_kineme.T 
    for num in range(0, num_chunk):
        data_chunk = final_data_file[num*(chunk_time-1):((num+1)*(chunk_time-1))]
        data_chunk = data_chunk.to_numpy()
        data_list.append(data_chunk.flatten())              
    return  data_list

**Creating AU (Action Units) Feature Chunks**

In [ ]:
def max_encoding(input_file_au,threshold,chunk_size,total_chunks):
    # print("AU chunks")
    count = 0
    duration = 0
    overlap = 10
    complete_vec = []
    intensity_var = threshold  
    read_data = np.array(input_file_au) 
    for c in range(0,total_chunks,1):           #total_chunks= 10 , c= 0 
        vector = []
        for j in range(0, chunk_size-1, 1): #0 1 2 0 20 10 30 20 40
            # print("Chunkk number for this file " + str(j))
            # print("chunk dim "+ str(duration) + " "+ str(duration+20))
            max_pool = [] 
            for i in range(5, 22, 1):
                max_value = np.max(read_data[duration:duration+20, i])
                if max_value <= intensity_var:
                    max_value = 0
                else:
                    max_value = 1
                max_pool.append(max_value)
            duration = duration + overlap
            vector.append(max_pool)
        vec_flat = np.array(vector).flatten()
        complete_vec.append(vec_flat)
    return complete_vec


**Creating Audio Feature Chunks**

In [ ]:
def chunks_formation(f, chunk_time, size_of_feature_set,num_chunks):
    # print("Audio chunks")
    data_list = []
    csv_file   = pd.read_csv(f, header=None)
    i = 0          
    entire_file_mat = []     
    while i < csv_file.shape[1]:
        new_frame = csv_file.loc[:, i:i+87]
        # print("Frame dimension "+ str(i) + " " + str(i+87))
        i = i + 44
        avg_value = new_frame.mean(axis=1)
        file_mat = pd.concat([avg_value], axis=1, ignore_index=True)
        file_mat = file_mat.to_numpy().flatten()
        file_mat = pd.DataFrame(file_mat)
        entire_file_mat = np.concatenate((entire_file_mat, file_mat), axis=None)
        new_array = entire_file_mat
        value_to_be_minus = new_array.shape[0]-(2*size_of_feature_set)
        new_array = new_array[0:value_to_be_minus] #To handle extra 2 sec data
    for num in range(0, num_chunks):
        data_chunk = new_array[num*(chunk_time-1)*size_of_feature_set:((num+1)*(chunk_time-1)*size_of_feature_set)]
        data_list.append(data_chunk)  
    data_array = np.array(data_list) 
    my_df = pd.DataFrame(data_array)
    Data = my_df.fillna(method='ffill')
    Data = Data.fillna(method='bfill')
    Data = np.asarray(Data)
    return Data

def chunks_formation_test(file_path, chunk_time, size_of_feature_set,scaler,num_chunks):
    f = file_path
    data_list = []
    csv_file   = pd.read_csv(f, header=None)
    i = 0          
    entire_file_mat = []
    while i < csv_file.shape[1]:
        new_frame = csv_file.loc[:, i:i+87]
        i = i + 44
        avg_value = new_frame.mean(axis=1)
        file_mat = pd.concat([avg_value], axis=1, ignore_index=True)
        file_mat = file_mat.to_numpy().flatten()
        file_mat = pd.DataFrame(file_mat)
        entire_file_mat = np.concatenate((entire_file_mat, file_mat), axis=None)
        new_array = entire_file_mat
        value_to_be_minus = new_array.shape[0]-(2*size_of_feature_set)
        new_array = new_array[0:value_to_be_minus] #To handle extra 2 sec data
    for num in range(0, num_chunks):
        data_chunk = entire_file_mat[num*(chunk_time-1)*size_of_feature_set:((num+1)*(chunk_time-1)*size_of_feature_set)]
        data_list.append(data_chunk)
    data_array = np.asarray(data_list)
    data_df= pd.DataFrame(data_array)
    Data = data_df.fillna(method='ffill')
    Data = Data.fillna(method='bfill')
    Data = np.asarray(Data)
    scaled_Data = scaler.transform(Data)
    Data_final = scaled_Data
    return Data_final

**Attention Fusion Model**

In [ ]:
# parameters
nNeuron = 12
chunk_size = 10
label_name = 'EyeContact'
seqLen = chunk_size-1
nKineme, nClass = 16, 1
nAction, nAudio = 17, 23
EPOCHS = 30
BATCH_SIZE = 32
threshold_action_units = 1.5

def split_tensor(X):
  s0, s1, s2 = tf.split(X, num_or_size_splits=3,axis = -1)
  return s0,s1,s2

def mul1(x):
  soft = x
  with tf.compat.v1.Session() as sess:
    T_onehot = tf.one_hot(tf.argmax(soft, 1), soft.shape[1])
  return T_onehot

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#left input LSTM
kineme_branch_input = Input(shape=(seqLen, nKineme), name="kineme_input")
kineme_branch_output = LSTM(nNeuron, activation = 'tanh', name="kineme_dense")(kineme_branch_input)
kineme_branch_norm = LayerNormalization( name = "kineme_normal")(kineme_branch_output)
#middle branch input
au_branch_input = Input(shape=(seqLen, nAction), name='action_input')
au_branch_output = LSTM(nNeuron, activation = 'tanh', name="action_dense")(au_branch_input)
au_branch_norm = LayerNormalization(name = "action_normal")(au_branch_output)
#right input LSTM
audio_branch_input = Input(shape=(seqLen, nAudio), name='audio_input')
audio_branch_output = LSTM(nNeuron, activation = 'tanh', name='audio_dense')(audio_branch_input)
audio_branch_norm = LayerNormalization(name = "audio_normal")(audio_branch_output)
#Merged Layer and calculating softmax score from the dense layer of the actual real embeddings 
merge_layer = concatenate([kineme_branch_output, au_branch_output, audio_branch_output], name='Concatenate_modalities')
merge_dense = Dense(nNeuron, activation='tanh', name='concatenate_dense')(merge_layer)
softmax_output = Dense(3, activation='softmax', name='softmax_score')(merge_dense)

kineme_val, au_val, audio_val = Lambda(split_tensor)(softmax_output)

score_kin_mul = multiply([kineme_branch_norm, kineme_val], name='Concatenate_score_mul_k')
score_au_mul = multiply([au_branch_norm, au_val], name='Concatenate_score_mul_au')
score_audio_mul = multiply([audio_branch_norm, audio_val], name='Concatenate_score_mul_aud')

#for concatenate attention
addition_layer = Add()([score_kin_mul, score_au_mul, score_audio_mul])
final_model_output = Dense(nClass, activation='sigmoid')(addition_layer)
final_model = Model(inputs=[kineme_branch_input, au_branch_input, audio_branch_input], outputs=final_model_output,name='Final_output')

opt = keras.optimizers.Adam(learning_rate=0.01)
final_model.compile(optimizer = opt, loss = 'binary_crossentropy',metrics=['accuracy'])
final_model.summary()

**Data Files**

In [ ]:
#path to all the kineme files
file_list = sorted(glob.glob('/content/drive/MyDrive/Research_Related/Ongoing_Research/Kineme_Project/Kineme_Files/MIT/VL/AU_10fps/*.csv'))
label_data = pd.read_csv('/content/drive/MyDrive/Research_Related/Ongoing_Research/Kineme_Project/Kineme_Files/MIT/VL/labels_traits_minmax_norm.csv')
label = label_data[label_name]
f = np.array(file_list).reshape(-1,1) #All data files
label = pd.to_numeric(label, downcast='integer') #Float to integer conversion
label = label.to_numpy().reshape(-1,1)  
file_to_work = np.concatenate((f, label),axis=1)   #File plus label in single array

test_loss=[]
test_acc=[]
train_loss = []
train_acc = []
fi_weighted=[]
fi_macro=[]
val_loss = []
val_acc = []
inter_output_train = np.zeros((1,3))
inter_output_test = np.zeros((1,3))
list_file_test_set = []
# print(inter_output_test)


**Train and Test**

In [ ]:
count = 1
random_state = 42
rkf = RepeatedKFold(n_splits=10, n_repeats=5, random_state=random_state)      #repeat kfold function
for train_idx, test_idx in rkf.split(file_to_work):
    print("@@@@@@@@ On the iteration " + str(count) + " @@@@@@@@")
    train_features, test_features = file_to_work[train_idx], file_to_work[test_idx]
    file_path = train_features[:,0].tolist()  #File list for chunk preparation
    labels = train_features[:,1]  #labels for chunk preparation
    file_name = Path(file_path[0]).stem 
    #paths for files
    outer_directory  = '/content/drive/MyDrive/Research_Related/Ongoing_Research/Kineme_Project/Kineme_Files/MIT/VL'
    AU_path = outer_directory + '/AU_10fps/' + file_name + '.csv'
    Kine_Path = outer_directory + '/Kinemes/' + file_name + '.csv'
    Audio_path = outer_directory + '/MIT_Data_CSV/' + file_name +'_audio.csv'
    #creation of data frames with the initial csv paths of kineme and AU and label; 
    AU1 = pd.read_csv(AU_path)
    Kin1 = pd.read_csv(Kine_Path, header=None)
    label = labels[0]
    chun_au = int(AU1.shape[0]/((chunk_size)*10))
    chun_ki = int(Kin1.shape[1]/(chunk_size))
    if chun_au <= chun_ki:
        total_size = chun_au
    else:
        total_size = chun_ki
    kin_res1,label1 = data_preprocess(chunk_size,Kin1,label,total_size)
    au_res1 = max_encoding(AU1,threshold_action_units,chunk_size,total_size)
    audio_res1 = chunks_formation(Audio_path, chunk_size, nAudio,total_size)
    kin_res1 = np.array(kin_res1)
    au_res1 = np.array(au_res1)
    for i in range(0,len(file_path)-1,1):
        file_name = Path(file_path[i+1]).stem   
        AU_path = outer_directory + '/AU_10fps/' + file_name + '.csv'
        Kine_Path = outer_directory + '/Kinemes/' + file_name + '.csv'
        Audio_path = outer_directory + '/MIT_Data_CSV/' + file_name +'_audio.csv'
        AU2 = pd.read_csv(AU_path)
        Kin2 = pd.read_csv(Kine_Path, header=None)
        label2 = labels[i+1]
        chun_au = int(AU2.shape[0]/(chunk_size*10))
        chun_ki = int(Kin2.shape[1]/(chunk_size))
        if chun_au<=chun_ki:
            total_size = chun_au
        else:
            total_size = chun_ki
        kin_res2,label2 = data_preprocess(chunk_size,Kin2,label2,total_size)
        au_res2 = max_encoding(AU2,1.5,chunk_size,total_size)
        audio_res2 = chunks_formation(Audio_path, chunk_size, 23,total_size)
        #make array of results
        kin_res2 = np.array(kin_res2)
        au_res2 = np.array(au_res2)
        #Matrix merging
        kin_res1 = np.vstack((kin_res1,kin_res2))
        au_res1 = np.vstack((au_res1,au_res2))
        audio_res1 = np.vstack((audio_res1,audio_res2))
        label1.extend(label2)
        # print(file_name, total_size)

    final = np.concatenate((kin_res1.T, au_res1.T)).T
    scaler = preprocessing.StandardScaler().fit(audio_res1)
    final_audio = scaler.transform(audio_res1)
    final_label = np.array(label1)
    final_label = [float(i) for i in final_label]   #String to float conversion
    train_median = find_median(final_label)
    final_label = bin_labels(final_label)
    train_labels = np.array(final_label)
    train_kinemes = ks_encoding(final[:,0:seqLen], nKineme)
    train_action = final[:, seqLen:]
    train_action = train_action.reshape((train_action.shape[0], seqLen, nAction))
    train_audio = final_audio.reshape((final_audio.shape[0], seqLen, nAudio))
    zero_bias_history = final_model.fit([train_kinemes, train_action, train_audio], train_labels, epochs = EPOCHS, batch_size = 32, validation_split=0.1, callbacks=[callback])  #Fitting the model 
    print("Model Training is Done")

    #Process for testing
    test_data = test_features[:,0].tolist()  #test data
    test_labels = test_features[:,1]  #test actual label
    test_labels = [float(i) for i in test_labels]   #String to int conversion
    test_labels = bin_labels_test(test_labels, train_median)
    y_pred_video = []
    
    for i in range(0,len(test_data)):
        file_name = Path(test_data[i]).stem   
        AU_path = outer_directory + '/AU_10fps/' + file_name + '.csv'
        Kine_Path = outer_directory + '/Kinemes/' + file_name + '.csv'
        Audio_path = outer_directory + '/MIT_Data_CSV/' + file_name +'_audio.csv'

        AU2 = pd.read_csv(AU_path)
        Kin2 = pd.read_csv(Kine_Path, header=None)
        chun_au = int(AU2.shape[0]/((chunk_size)*10))
        chun_ki = int(Kin2.shape[1]/(chunk_size))
        if chun_au <= chun_ki:
            total_size = chun_au
        else:
            total_size = chun_ki
        kin_res2 = data_preprocess_test(chunk_size,Kin2,total_size)
        au_res2 = max_encoding(AU2,1.5,chunk_size,total_size)
        audio_res2 = chunks_formation_test(Audio_path, chunk_size, 23,scaler,total_size)

        #make array of results
        kin_res2 = np.array(kin_res2)
        au_res2 = np.array(au_res2)
        final_test = np.concatenate((kin_res2.T, au_res2.T)).T
        
        test_kinemes = ks_encoding(final_test[:,0:seqLen], nKineme)
        test_action = final_test[:, seqLen:]
        test_action = test_action.reshape((test_action.shape[0], seqLen, nAction))
        test_audio = audio_res2.reshape((audio_res2.shape[0], seqLen, nAudio))
        y_pred = final_model.predict([test_kinemes, test_action, test_audio])
        y_pred = ((y_pred >= 0.5)+0).ravel()
        y1 = majority_vote(y_pred)   #Voting for classification to change
        y_pred_video.append(y1)
        print("----------Files in test set")
        print(file_name, total_size)
        list_file_test_set.extend([file_name, total_size])
        intermediate_model_folds= Model(inputs=final_model.input ,outputs=final_model.get_layer(index = 8).output)
        inter_output_fold_test = intermediate_model_folds.predict([test_kinemes, test_action, test_audio])
        inter_output_fold_test = np.asarray(inter_output_fold_test)
        inter_output_test = np.row_stack((inter_output_test, inter_output_fold_test))
    acc_val = accuracy_score(test_labels, y_pred_video)
    test_acc.append(acc_val)
    print('Test accuracy:', acc_val)   
    train_acc.append(np.array(zero_bias_history.history['accuracy']).mean()) 
    val_acc.append(np.array(zero_bias_history.history['val_accuracy']).mean()) 
    f1_w_epoch = f1_score(test_labels, y_pred_video, average='weighted')
    f1_m_epoch = f1_score(test_labels, y_pred_video, average='macro')
    fi_weighted.append(f1_w_epoch)
    fi_macro.append(f1_m_epoch)